In [3]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/datasets

/content/drive/MyDrive/datasets


## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена:
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ.
  
  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```

1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size]
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import string

# Чтение данных из файла
with open('name_rus.txt', 'r', encoding='windows-1251') as f:
    names = f.read().splitlines()

# Создание словаря токенов
special_tokens = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']
all_chars = sorted(set(''.join(names))) + special_tokens
char2idx = {char: idx for idx, char in enumerate(all_chars)}
idx2char = {idx: char for char, idx in char2idx.items()}

# Преобразование имен в индексы
def encode_name(name, max_len=20):
    encoded = [char2idx['<SOS>']] + [char2idx.get(char, char2idx['<UNK>']) for char in name]
    encoded += [char2idx['<EOS>']]
    encoded += [char2idx['<PAD>']] * (max_len - len(encoded))
    return encoded[:max_len]

# Декодирование индексов обратно в имя
def decode_name(indices):
    return ''.join([idx2char[idx] for idx in indices if idx2char[idx] not in special_tokens])

# Создание датасета
class NameDataset(Dataset):
    def __init__(self, names, max_len=20):
        self.data = [encode_name(name, max_len) for name in names]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return torch.tensor(x), torch.tensor(y)

dataset = NameDataset(names)

In [6]:
import torch.nn as nn
import torch.optim as optim

# Модель LSTM
class NameGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=2):
        super(NameGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = out.reshape(-1, out.size(2))
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))

# Функция для генерации фамилий
def generate_name(model, max_len=20):
    model.eval()
    input_seq = torch.tensor([[char2idx['<SOS>']]])
    hidden = None
    generated_name = []

    for _ in range(max_len):
        output, hidden = model(input_seq, hidden)
        probs = torch.softmax(output, dim=-1)
        next_char_idx = torch.multinomial(probs[-1], num_samples=1)
        next_char = next_char_idx.item()

        if idx2char[next_char] == '<EOS>':
            break
        generated_name.append(idx2char[next_char])
        input_seq = next_char_idx.unsqueeze(0)

    return ''.join(generated_name)

# Гиперпараметры
embedding_dim = 32
hidden_dim = 128
num_layers = 2
vocab_size = len(char2idx)
batch_size = 64
epochs = 100
max_len = 20

# Подготовка данных
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Инициализация модели
model = NameGenerator(vocab_size, embedding_dim, hidden_dim, num_layers)
criterion = nn.CrossEntropyLoss(ignore_index=char2idx['<PAD>'])
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Генерация фамилий каждые 5 эпох
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for x, y in dataloader:
        optimizer.zero_grad()
        hidden = model.init_hidden(x.size(0))
        output, hidden = model(x, hidden)
        loss = criterion(output, y.view(-1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader):.4f}")

    # Генерация фамилий каждые 5 эпох
    if (epoch + 1) % 5 == 0:
        print("Generated names:")
        for _ in range(5):
            print(generate_name(model))

# Пример использования
print("Generated name:", generate_name(model))

Epoch 1, Loss: 3.1103
Epoch 2, Loss: 2.6773
Epoch 3, Loss: 2.4485
Epoch 4, Loss: 2.2828
Epoch 5, Loss: 2.1603
Generated names:
гккадя
донетка
локюонка
лиуью
гиляшиса
Epoch 6, Loss: 2.0950
Epoch 7, Loss: 2.0445
Epoch 8, Loss: 1.9792
Epoch 9, Loss: 1.9196
Epoch 10, Loss: 1.8825
Generated names:
тируша
фетюня
егуня
пэстьонка
малюха
Epoch 11, Loss: 1.8437
Epoch 12, Loss: 1.8192
Epoch 13, Loss: 1.7684
Epoch 14, Loss: 1.7353
Epoch 15, Loss: 1.7187
Generated names:
лада
мольша
моля
даня
амоныха
Epoch 16, Loss: 1.6754
Epoch 17, Loss: 1.6433
Epoch 18, Loss: 1.6247
Epoch 19, Loss: 1.5964
Epoch 20, Loss: 1.5684
Generated names:
денена
спатя
юлиан
ледий
боря
Epoch 21, Loss: 1.5445
Epoch 22, Loss: 1.5260
Epoch 23, Loss: 1.5149
Epoch 24, Loss: 1.4938
Epoch 25, Loss: 1.4787
Generated names:
кизюша
геруша
сева
оман
фешуся
Epoch 26, Loss: 1.4550
Epoch 27, Loss: 1.4433
Epoch 28, Loss: 1.4297
Epoch 29, Loss: 1.4112
Epoch 30, Loss: 1.4048
Generated names:
максюша
няха
брина
анокулка
торюха
Epoch 31, Loss:

## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет.

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова)
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Загрузка текста из файла
with open("dostoevsky_cut.txt", "r", encoding="utf-8") as file:
    text = file.read().lower()

# Создание словаря токенов
chars = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(chars)}
idx2char = {idx: char for idx, char in enumerate(chars)}
vocab_size = len(chars)

# Преобразование текста в последовательность индексов
def text_to_indices(text):
    return [char2idx[char] for char in text]

# Разделение текста на последовательности
def create_sequences(data, seq_len):
    sequences = []
    for i in range(0, len(data) - seq_len, seq_len):
        sequences.append(data[i:i + seq_len + 1])  # +1 для правильного ответа
    return sequences

# Создание датасета
class TextDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        x = torch.tensor(sequence[:-1], dtype=torch.long)
        y = torch.tensor(sequence[1:], dtype=torch.long)
        return x, y

# Гиперпараметры
seq_len = 100  # Длина последовательности
batch_size = 64
embedding_dim = 128
hidden_dim = 256
num_layers = 2
epochs = 20
learning_rate = 0.001
clip_value = 5  # Обрезка градиентов

# Подготовка данных
data = text_to_indices(text)
sequences = create_sequences(data, seq_len)
dataset = TextDataset(sequences)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Модель RNN
class TextGenerator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(TextGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        out, hidden = self.rnn(x, hidden)
        out = out.reshape(-1, out.size(2))
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, batch_size, self.hidden_dim))

# Инициализация модели
model = TextGenerator(vocab_size, embedding_dim, hidden_dim, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for x, y in dataloader:
        optimizer.zero_grad()
        hidden = model.init_hidden(x.size(0))
        output, hidden = model(x, hidden)
        loss = criterion(output, y.view(-1))
        loss.backward()
        # Обрезка градиентов
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader):.4f}")

# Функция для генерации текста
def generate_text(model, prime, length=200, temperature=0.8):
    model.eval()
    hidden = None
    prime_indices = text_to_indices(prime)
    input_seq = torch.tensor([prime_indices], dtype=torch.long)

    # Пропуск prime через модель для получения скрытого состояния
    for char in prime_indices[:-1]:
        _, hidden = model(input_seq, hidden)
        input_seq = torch.tensor([[char]], dtype=torch.long)

    generated_text = prime
    input_seq = torch.tensor([[prime_indices[-1]]], dtype=torch.long)

    # Генерация текста
    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        probs = torch.softmax(output / temperature, dim=-1).squeeze()
        next_char_idx = torch.multinomial(probs, num_samples=1).item()
        next_char = idx2char[next_char_idx]
        generated_text += next_char
        input_seq = torch.tensor([[next_char_idx]], dtype=torch.long)
    return generated_text

Epoch 1, Loss: 3.0202
Epoch 2, Loss: 2.4097
Epoch 3, Loss: 2.1767
Epoch 4, Loss: 2.0312
Epoch 5, Loss: 1.9299
Epoch 6, Loss: 1.8534
Epoch 7, Loss: 1.7946
Epoch 8, Loss: 1.7463
Epoch 9, Loss: 1.7067
Epoch 10, Loss: 1.6725
Epoch 11, Loss: 1.6439
Epoch 12, Loss: 1.6184
Epoch 13, Loss: 1.5966
Epoch 14, Loss: 1.5753
Epoch 15, Loss: 1.5579
Epoch 16, Loss: 1.5424
Epoch 17, Loss: 1.5278
Epoch 18, Loss: 1.5119
Epoch 19, Loss: 1.4986
Epoch 20, Loss: 1.4858


In [8]:
# Пример генерации текста
prime_text = "скучно"
generated_text = generate_text(model, prime_text, length=300)
print(generated_text)

скучности, — я неужели и начиналось виде подроднулся ко годовой и вергновными николаевны и тогда так были по-настого взглядо-собидно, что ему при чего-нибудь заключить к нам любопытских словом, что позволите, как теперь ангельчика. начали бы только что вас скажу, только после как к вам девушки какий-то сч
